### 1. Utilitários

In [13]:
import pandas as pd
import os

In [14]:
# Lista de 3 valores:
# Caminho dos arquivo no diretório. (str)
# Nome para Dataframe. (str)
# Nome para subtituir coluna "Total" de cada arquivo. (str)
dados = [
    (os.path.join("..", "Área de Culturas Permanentes (hectares)", "Área colhida permanente 1999-2020.csv"), "df_permanente", "Culturas P. Colhidas (hec)"),
    (os.path.join("..", "Producao-Milhos", "Produção - milho(R$).xls"), "df_milho", "Producao de Milho (R$)"),
    (os.path.join("..", "Producao-Soja", "produçãosoja.xls"), "df_soja", "Producao de Soja (R$)"),
    (os.path.join("..", "Despesa Orçamentária (R$)", "Despesa por função - gestão ambiental, agricultura e organização agrária - empenhada - estadual.xls"), "df_despesa", "Despesa Orçamentária (R$)"),
    (os.path.join("..", "Focos de Incendio", "firefoci.csv"), "df_incendio", "Focos de Incêndio"),
    (os.path.join("..", "Área de Culturas Temporárias (hectares)", "Área colhida temporária 1999-2020.csv"), "df_temporaria", "Culturas T. Colhidas (hec)"),
    (os.path.join("..", "Área Plantada ou Colhida (hectares)", "Área plantada ou destinada à colheita - total.xls"), "df_plantada_colhida", "Área Plantada ou Colhida (hec)"),
    (os.path.join("..", "PIB", "PIB 1999-2020.csv"), "df_pib", "PIB Estadual"),
    (os.path.join("..", "Valor da Produção Agrícola (R$)", "Valor de Produção 1999-2020.csv"), "df_producao", "PIB Agrícula Estadual") 
]

# Lista dos estados que queremos filtrar
estados = ["Acre", "Amazonas", "Amapá", "Maranhão", "Pará", "Roraima", "Tocantins", "Rondônia", "Mato Grosso"] 

# Lista de anos como strings
anos_str = [str(year) for year in range(1999, 2021)]

### 2. Pré-processamento

O objetivo final é unir os valores anuais de 1999 a 2020 de cada variável estadual dos 9 estados da região em uma tabela única, para futura implementação de um algoritmo de machine learning.

In [41]:
for arquivo, nome_df, coluna in dados:
    try:
        # Ler arquivos .csv com separador ',', se não, ler arquivo .xls
        try:
            df = pd.read_csv(arquivo, sep=",")
        except:
            df = pd.read_excel(arquivo)

        # Normalizar os nomes das colunas (remover espaços extras)
        df.columns = df.columns.str.strip()

        # Configurar Pandas para exibir números inteiros completos
        pd.options.display.float_format = '{:.2f}'.format
        
        # Remover colunas "Código" e "Sigla", se existirem
        colunas_para_remover = [col for col in ["Codigo", "Código", "Sigla"] if col in df.columns]
        if colunas_para_remover:
            df = df.drop(columns=colunas_para_remover)
            print(f"🗑️ Colunas removidas de {nome_df}: {colunas_para_remover}")

        # Filtrar os estados desejados
        if "Estado" in df.columns:
            df = df[df["Estado"].isin(estados)]
            print(f"🌎 Filtrados apenas os estados desejados em {nome_df}")

        # Armazenar na variável correta
        globals()[nome_df] = df
        print(f"✔ {arquivo} carregado como {nome_df} \n")
        
    except Exception as e:
        print(f"Erro ao ler {arquivo}: {e}")

# Exibir quais DataFrames foram carregados
print("\n📂 DataFrames disponíveis:", list(nome_df for _, nome_df, _ in dados))

🗑️ Colunas removidas de df_permanente: ['Codigo', 'Sigla']
🌎 Filtrados apenas os estados desejados em df_permanente
✔ ..\Área de Culturas Permanentes (hectares)\Área colhida permanente 1999-2020.csv carregado como df_permanente 

🗑️ Colunas removidas de df_milho: ['Codigo', 'Sigla']
🌎 Filtrados apenas os estados desejados em df_milho
✔ ..\Producao-Milhos\Produção - milho(R$).xls carregado como df_milho 

🗑️ Colunas removidas de df_soja: ['Codigo', 'Sigla']
🌎 Filtrados apenas os estados desejados em df_soja
✔ ..\Producao-Soja\produçãosoja.xls carregado como df_soja 

🗑️ Colunas removidas de df_despesa: ['Codigo', 'Sigla']
🌎 Filtrados apenas os estados desejados em df_despesa
✔ ..\Despesa Orçamentária (R$)\Despesa por função - gestão ambiental, agricultura e organização agrária - empenhada - estadual.xls carregado como df_despesa 

✔ ..\Focos de Incendio\firefoci.csv carregado como df_incendio 

🗑️ Colunas removidas de df_temporaria: ['Código', 'Sigla']
🌎 Filtrados apenas os estados dese

##### 3. Transformação formato Wide para Long

Os DataFrames estão no formato longo, com 22 colunas representando os anos de 1999 a 2020, exceto no Data frame "Focos de incêndio", em que as colunas representam os 9 estados.
A função .melt() permite transformar essas colunas em linhas, facilitando a análise e manipulação dos dados.

In [16]:
df_incendio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Ano          22 non-null     int64  
 1   Acre         22 non-null     float64
 2   Amapá        22 non-null     float64
 3   Amazonas     22 non-null     float64
 4   Maranhão     22 non-null     float64
 5   Mato Grosso  22 non-null     float64
 6   Pará         22 non-null     float64
 7   Rondônia     22 non-null     float64
 8   Roraima      22 non-null     float64
 9   Tocantins    22 non-null     float64
dtypes: float64(9), int64(1)
memory usage: 1.8 KB


In [17]:
for arquivo, nome_df, coluna in dados:
    df = globals()[nome_df]
    try:
        df = df.melt(id_vars=['Estado'], value_vars=anos_str, var_name='Ano', value_name= f'{coluna}')
    except:
        #alterações df_incendio
        df["Ano"] = df['Ano'].astype(str)
        df = df.melt(id_vars=["Ano"], value_vars=estados, var_name="Estado", value_name= f'{coluna}')
    globals()[nome_df] = df

In [18]:
df_incendio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Ano                198 non-null    object 
 1   Estado             198 non-null    object 
 2   Focos de Incêndio  198 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.8+ KB


##### 3. Merge do DataFrame final

In [19]:
df_geral = df_permanente.merge(df_milho, how='outer', on=["Ano", "Estado"])\
                            .merge(df_soja, how='outer', on=["Ano", "Estado"] )\
                            .merge(df_despesa, how='outer', on=["Ano", "Estado"])\
                            .merge(df_temporaria, how='outer', on=["Ano", "Estado"])\
                            .merge(df_plantada_colhida, how='outer', on=["Ano", "Estado"])\
                            .merge(df_pib, how='outer', on=["Ano", "Estado"])\
                            .merge(df_producao, how='outer', on=["Ano", "Estado"])\
                            .merge(df_incendio, how='outer', on=["Ano", "Estado"])
df_geral

,Estado,Ano,Culturas P. Colhidas (hec),Producao de Milho (R$),Producao de Soja (R$),Despesa Orçamentária (R$),Culturas T. Colhidas (hec),Área Plantada ou Colhida (hec),PIB Estadual,PIB Agrícula Estadual,Focos de Incêndio
0,Acre,1999,9864,18904.94,135.89,16480351.00,81922,91787,1866542.32,280900.59,347.00
1,Amapá,1999,1283,663.59,NaN,2869221.75,6903,8235,2128307.03,54019.05,101.00
2,Amazonas,1999,61397,12572.44,875.29,17685970.41,150055,213964,13933920.08,623336.20,1048.00
3,Maranhão,1999,28896,96560.50,243966.36,15239850.98,1130797,1180275,10308256.16,1174349.20,8712.00
4,Mato Grosso,1999,91480,287455.01,3470093.09,43232165.00,4259560,4445242,12364904.96,6462377.66,44307.00
...,...,...,...,...,...,...,...,...,...,...,...
193,Mato Grosso,2020,37589,10090285.97,22939332.03,555308252.18,17149919,17188641,178649564.00,41799931.48,47708.00
194,Pará,2020,651600,346956.98,1428548.90,380705994.45,1310866,1968093,215935604.00,7981858.68,38603.00
195,Rondônia,2020,83545,396740.36,884772.42,210685707.64,721826,813919,51598741.00,2196163.25,11145.00
196,Roraima,2020,10007,47137.95,162262.75,67296396.75,84045,94052,16024276.00,422452.65,1930.00


##### 4. Normalização por área

Ajustar os valores de cada registro de acordo com a extensão territorial de seu respectivo estado

In [20]:
arquivo_area = os.path.join("Extensão Territorial", "IBGE_area_geografica_98-22.csv")
df_area = pd.read_csv(arquivo_area, sep=";", skiprows=1)

In [29]:
df_area

,Sigla,Código,Estado,1998,2000,2010,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 16
0,AC,12,Acre,"153149,9","152521,9","164123,04","164123,739","164123,739","164123,712","164123,737","164123,737","164123,738","164123,964","164173,431","164173,431","164173,431",NaN
2,AM,13,Amazonas,"1577820,2","1570947,1","1559159,148","1559148,889","1559148,89","1559149,074","1559146,876","1559146,876","1559168,117","1559167,889","1559167,878","1559167,879","1559255,883",NaN
3,AP,16,Amapá,"143453,7","142815,8","142828,521","142828,52","142828,52","142828,523","142828,521","142828,521","142470,762","142470,762","142470,762","142470,762","142470,762",NaN
9,MA,21,Maranhão,"333365,6","331918,6","331937,45","331936,948","331936,948","331936,955","331936,949","331936,949","329642,17","329642,182","329651,495","329651,495","329651,495",NaN
12,MT,51,Mato Grosso,"906806,9","903385,2","903366,192","903378,292","903378,292","903198,091","903202,446","903202,445999999","903206,997","903207,019","903207,05","903207,047","903208,362",NaN
13,PA,15,Pará,"1253164,5","1247703,1","1247954,666","1242966,084","1242966,084","1242967,145","1242967,002","1242967,002","1240771,069","1240882,562","1240882,471","1240882,471","1240882,471",NaN
20,RO,11,Rondônia,"238512,8","237564,6","237590,547","237590,543","237590,543","237765,376","237765,293","237765,293","237765,233","237765,24","237765,347","237765,347","237754,173",NaN
21,RR,14,Roraima,"225116,1","224118,1","224300,506","224303,187","224303,187","224301,08","224300,805","224300,805","224273,831","223644,527","223644,527","223644,527","223644,527",NaN
26,TO,17,Tocantins,"278420,7","277297,4","277720,52","277720,569","277720,569","277720,567","277720,412","277720,412","277720,404","277466,763","277423,63","277423,63","277423,631",NaN


In [34]:
#Seleciona os estados de interesse
df_area = df_area.query("@estados in Estado")

#Seleciona os dados mais recentes (2022)
df_area22 = df_area[["Estado", "2022"]]

#Altera tipo da Coluna 2022 para float
df_area22["2022"] = df_area22["2022"].str.replace(",", ".", regex=True)  # Troca vírgula por ponto
df_area22["2022"] = pd.to_numeric(df_area22["2022"], errors="coerce")
df_area22 = df_area22.rename(columns={"2022" : "Área"})
df_area22

C:\Users\tsuki\AppData\Local\Temp\ipykernel_15296\798041545.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_area22["2022"] = df_area22["2022"].str.replace(",", ".", regex=True)  # Troca vírgula por ponto
C:\Users\tsuki\AppData\Local\Temp\ipykernel_15296\798041545.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_area22["2022"] = pd.to_numeric(df_area22["2022"], errors="coerce")


,Estado,Área
0,Acre,164173.43
2,Amazonas,1559255.88
3,Amapá,142470.76
9,Maranhão,329651.49
12,Mato Grosso,903208.36
13,Pará,1240882.47
20,Rondônia,237754.17
21,Roraima,223644.53
26,Tocantins,277423.63


teste

In [35]:
df_com_area = df_geral.merge(df_area22, how='outer', on=["Estado"])
df_com_area.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Estado                          198 non-null    object 
 1   Ano                             198 non-null    object 
 2   Culturas P. Colhidas (hec)      198 non-null    int64  
 3   Producao de Milho (R$)          198 non-null    float64
 4   Producao de Soja (R$)           176 non-null    float64
 5   Despesa Orçamentária (R$)       198 non-null    float64
 6   Culturas T. Colhidas (hec)      198 non-null    int64  
 7   Área Plantada ou Colhida (hec)  198 non-null    int64  
 8   PIB Estadual                    198 non-null    float64
 9   PIB Agrícula Estadual           198 non-null    float64
 10  Focos de Incêndio               198 non-null    float64
 11  Área                            198 non-null    float64
dtypes: float64(7), int64(3), object(2)
m

In [37]:
colunas = [col for _, _, col in dados]  # Lista com os nomes das colunas
df_com_area[colunas] = df_com_area[colunas].div(df_com_area["Área"].replace(0, 1), axis=0)


In [38]:
df_com_area

,Estado,Ano,Culturas P. Colhidas (hec),Producao de Milho (R$),Producao de Soja (R$),Despesa Orçamentária (R$),Culturas T. Colhidas (hec),Área Plantada ou Colhida (hec),PIB Estadual,PIB Agrícula Estadual,Focos de Incêndio,Área
0,Acre,1999,0.06,0.12,0.00,100.38,0.50,0.56,11.37,1.71,0.00,164173.43
1,Acre,2000,0.07,0.15,NaN,175.62,0.57,0.64,13.12,1.91,0.00,164173.43
2,Acre,2001,0.06,0.13,NaN,142.60,0.56,0.62,15.08,1.46,0.01,164173.43
3,Acre,2002,0.08,0.18,NaN,250.05,0.55,0.69,18.10,1.64,0.04,164173.43
4,Acre,2003,0.09,0.22,NaN,231.92,0.58,0.69,20.57,1.86,0.06,164173.43
...,...,...,...,...,...,...,...,...,...,...,...,...
193,Tocantins,2016,0.02,0.74,4.40,787.49,4.31,4.44,113.85,7.60,0.05,277423.63
194,Tocantins,2017,0.01,0.83,5.04,818.50,4.61,4.62,122.95,8.20,0.06,277423.63
195,Tocantins,2018,0.05,0.84,5.52,833.75,4.86,4.91,128.56,9.05,0.03,277423.63
196,Tocantins,2019,0.05,1.08,5.68,915.18,5.09,5.14,141.86,9.65,0.05,277423.63


In [ ]:
df_geral.to_csv("tabela_geral", index = False, encoding="utf-8")
path = os.getcwd()

print("📂 Dataframe salvo em: ", path) 